In [2]:
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector
from multivargrid import MultiVarGrid
from copy import deepcopy
import random

In [3]:
starting_health = 0.5
hunger_rate = 0.1
birth_threshold = 0.8

In [22]:
class Fish(Agent):
    def __init__(self, unique_id, model, variant) -> None:
        super().__init__(unique_id, model)
        self.variant = variant
        self.health= starting_health
    
    def step(self) -> None:
        #   Check if Fish Lives or Gives Birth
        self.health-= hunger_rate
        # if self.hunger > birth_threshold:
        #     self.hunger = self.hunger / 2
        #     self.give_birth()
        # elif self.hunger <= 0 :
            # pass
        #     self.die()
        #     return
        # Check oxygen levels 

        self.move_to_food()
        # if self.check_for_food():
        #     self.eat()

    def move(self) -> None:
        possible_steps = self.model.grid.get_neighborhood(
            self.pos, moore=True, include_center=False
        ) 
        new_positon = self.random.choice(possible_steps)
        self.model.grid.move_agent(self ,new_positon)

    def move_to_food(self) -> None:
        """Locate the nearest source of food and move towards it"""
        self.locate_food()
        #   move

    def locate_food(self) -> set:
        """Search the range around your position and find food"""
        #   Get coordinates of surrounding cells
        possible_steps = self.model.grid.get_neighborhood(
            self.pos, moore=True, include_center=True) 
        #   Get algae density of surrounding cells
        algae_grid = deepcopy(self.model.variable_grid.return_variable_grid('algae'))
        neighbor_algae_value = {coord : algae_grid[coord[0]][coord[1]] for coord in possible_steps} 
        #   Stochastically determine next move
        # next_move = random.choices(list(neighbor_algae.keys()), list(neighbor_algae.values()))
        #   Deterministically determine next move
        highest_algae_value = neighbor_algae_value[self.pos]
        next_move = self.pos
        for neighbor in neighbor_algae_value:
            if neighbor_algae_value[neighbor] >= highest_algae_value:
                highest_algae_value = neighbor_algae_value[neighbor]
                print(neighbor_algae_value[neighbor])
                next_move = neighbor
                print(next_move)
                
        return next_move

    def check_for_food(self):
        """Check whether current cell has algae"""
        algae_grid = deepcopy(self.model.variable_grid.return_variable_grid('algae'))
        current_cell_algae_value = algae_grid[self.pos[0]][self.pos[1]]
        if current_cell_algae_value > 0:
            return True
        else:
            return False

    def get_food_intake(self):
        """Gauge how much algae can be consumed""" 
        

In [5]:
class Lake(Model):
    """A model with some number of agents."""

    def __init__(self, N, width, height, agent_variant):
        self.num_agents = N
        self.grid= MultiGrid(width, height, False)
        self.schedule = RandomActivation(self)
        self.agent_variant = agent_variant

        #   Create Grid Storing all Non-Agent Variables
        lake_variables = {'algae' : random.uniform(0, 1), 'oxygen' : random.uniform(0.6, 1)}
        self.variable_grid = MultiVarGrid(width, height, lake_variables)

        # Create agents
        for i in range(self.num_agents):
            a = Fish(i, self, agent_variant)
            self.schedule.add(a)
            # Add the agent to a random grid cell
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
            self.grid.place_agent(a, (x, y))

    def step(self):
        self.schedule.step()


In [20]:
lake = Lake(1, 3, 3, None)

for i in range(1,2):
    lake.step()
    

0.4565871631784809
(0, 0)
0.4565871631784809
(0, 1)
0.4565871631784809
(0, 2)
0.4565871631784809
(1, 0)
0.4565871631784809
(1, 1)
0.4565871631784809
(1, 2)


In [ ]:
from mesa.visualization.modules import CanvasGrid
from mesa.visualization.ModularVisualization import ModularServer


def agent_portrayal(agent):
    portrayal = {"Shape": "circle",
                 "Filled": "true",
                 "Layer": 0,
                 "Color": "red",
                 "r": 0.5}
    return portrayal

grid = CanvasGrid(agent_portrayal, 10, 10, 500, 500)
server = ModularServer(Lake,
                       [grid],
                       "Lake Model",
                       {"N":100, "width":10, "height":10, "agent_variant" : None })
server.port = 8521 # The default
server.launch()

Interface starting at http://127.0.0.1:8521


OSError: [Errno 98] Address already in use